In [2]:
from utils.mypytable import MyPyTable 
import utils.plot_utils as plot_utils
import importlib
import utils.myclassifiers
importlib.reload(utils.myclassifiers)
from utils.myclassifiers import MyDecisionTreeClassifier
from utils.myclassifiers import MyKNeighborsClassifier
from utils.myclassifiers import MyDummyClassifier
from utils.myclassifiers import MyNaiveBayesClassifier

import numpy as np # Added import statement
import utils.myevaluation
importlib.reload(utils.myevaluation)
import utils.myevaluation as myevaluation
from itertools import chain
import utils.myutils as utils
importlib.reload(utils)
importlib.reload(plot_utils)
table = MyPyTable()
file_name = 'Skyserver_SQL4_21_2022 12_41_18 AM.csv'
table.load_from_file('input_folder/' + file_name)

In [10]:
# Data Cleaning 
# Removing all zWarning that are not (0 - OK)
values, counts = utils.get_frequencies(table.data, 14)
print("zwarning values:", values)
print("zWarning counts:", counts) # Removing only 36 instances

def row_indexes_to_drop(table, col_idx):
    row_indexes = []
    row_indexes = [i for i in range(len(table)) if table[i][col_idx] != 0]

    return row_indexes

row_idxs = row_indexes_to_drop(table.data, 14)
print(row_idxs, len(row_idxs))
print(len(table.data))
table.drop_rows(row_idxs)
print(len(table.data))
# Takes 34s to remove 36 instances

zwarning values: [0.0, 4.0, 16.0, 20.0, 64.0, 128.0]
zWarning counts: [19964, 9, 20, 1, 4, 2]
[1113, 1491, 1936, 2575, 3320, 3387, 3778, 4710, 4950, 5129, 6709, 8001, 9710, 9868, 10027, 10754, 11221, 12218, 12685, 12830, 13131, 13183, 13438, 14085, 14245, 14665, 14910, 15606, 15730, 16165, 16388, 16471, 17324, 18428, 19176, 19680] 36
20000
19964


In [21]:
# Removing all the unphysical fluxes
def row_idx_flux(table, col_idxs):
    row_indexes = []
    
    for i in range(len(col_idxs)):
        for j in range(len(table)):
            if i not in row_indexes:
                if table[j][col_idxs[i]] >= (1*10**5) or table[j][col_idxs[i]] <= (-1*10**5):
                    row_indexes.append(j)

    return row_indexes

col_idxs = list(range(17, 28 + 1)) # Plus to account for exclusivity (17 start of fluxes)

row_flux =  row_idx_flux(table.data, col_idxs) # Identifies 18 rows with unphysical fluxes
print(len(table.data))
table.drop_rows(row_flux)
print(len(table.data))
# Takes 32s to remove 18 instances

19948
19948


In [20]:
# Ensuring erroneous fluxes removed 
summary_table = table.compute_summary_statistics(['p_el', 'p_cw', 'p_acw', 'p_edge', 'p_dk', 'p_mg', 'ra', 'dec', 'z', 'zWarning', 'waveMin', 'plateSN2', 'h_delta_flux', 'h_gamma_flux', 'oiii_4363_flux', 'h_beta_flux', 'oiii_4959_flux', 'oiii_5007_flux', 'hei_5876_flux', 'oi_6300_flux', 'h_alpha_flux', 'nii_6548_flux', 'sii_6717_flux', 'sii_6731_flux'])
summary_table.pretty_print()

attribute                   min           max            mid           avg       median
--------------  ---------------  ------------  -------------  ------------  -----------
p_el                 0               1              0.5          0.554646      0.604
p_cw                 0               1              0.5          0.0685233     0.014
p_acw                0               1              0.5          0.0754724     0.019
p_edge               0               1              0.5          0.200611      0.129
p_dk                 0               0.974          0.487        0.0701847     0.053
p_mg                 0               0.957          0.4785       0.0304911     0
ra                   0.00647288    359.853        179.93       202.561       189.056
dec                 -3.74796        66.3173        31.2847       4.7126       -0.289632
z                    0.0100696       0.397336       0.203703     0.105953      0.097698
zWarning             0               0              0    

## Next Steps 
1. Remove particularly weak emission lines (need to talk to Dr. Aver about this one)
1. Normalizaion with h_beta_flux 